In [1]:
%matplotlib inline
%load_ext memory_profiler

import os
import time
import datetime
import sys
import pickle
import scanpy as sc
from scipy import sparse
import numpy as np
from sklearn.model_selection import train_test_split

sc.settings.verbosity=2

#### Load AnnData obj

In [2]:

# fps
sdfp = '/home/ngr4/project/sccovid/shared/data/processed/'
pfp = '/home/ngr4/project/sccovid/results/'
pdfp = '/home/ngr4/project/sccovid/data/processed/'
sc.settings.figdir = pfp

def loader(fname,fpath,backed=None) : 
    start = time.time()
    adata = sc.read_h5ad(filename=os.path.join(fpath,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    return adata

def writer(fname,fpath,AnnData) :
    start = time.time()
    AnnData.write(os.path.join(fpath,fname))
    print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to save data'.format(time.time()-start))
    

if True :
    # load personal
    fname='scv2_200428.h5ad'
    %memit adata = loader(fname,pdfp)

    


loaded @200601.11:32:33
took 21.08-s to load data
peak memory: 18413.05 MiB, increment: 18229.24 MiB


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/anndata/compat/__init__.py:161: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/anndata/compat/__init__.py:161: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  FutureWarning,


In [3]:
pfp = '/home/ngr4/project/edge_feat/results'
def pklthat(gdata, fname, fpath=pdfp): 
    with open(os.path.join(fpath,fname),'wb') as f :
        pickle.dump(gdata, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()
        
pklthat(adata.var_names.to_list(), 'hbec_feat_names.pkl', fpath=pfp)

# Induction

Sample 1/3 of the data randomly, grab labels

In [3]:
# split the data AND kick out mock infected cells as these are noise
idx_train, idx_test = train_test_split(adata.obs.index, train_size=0.7)
tdata = sc.AnnData(X=adata[(adata.obs.index.isin(idx_train)) & (~((adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==1))),:].X,
                   obs=adata[(adata.obs.index.isin(idx_train)) & (~((adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==1))),:].obs)
idx_val, idx_test = train_test_split(idx_test, train_size=0.5)
val = sc.AnnData(X=adata[(adata.obs.index.isin(idx_val)) & (~((adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==1))),:].X, 
                  obs=adata[(adata.obs.index.isin(idx_val)) & (~((adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==1))),:].obs)
test = sc.AnnData(X=adata[(adata.obs.index.isin(idx_test)) & (~((adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==1))),:].X, 
                  obs=adata[(adata.obs.index.isin(idx_test)) & (~((adata.obs['Condition']=='Mock') & (adata.obs['scv2_10+']==1))),:].obs)

def graph_pp(AnnData, bbknn=False):
    sc.tl.pca(AnnData, n_comps=100)
    if bbknn:
        sc.external.pp.bbknn(AnnData) # use default params
    else:
        sc.pp.neighbors(AnnData, n_pcs=100, n_neighbors=30)
    return AnnData

# make graph
tdata = graph_pp(tdata, bbknn=True)
val = graph_pp(val, bbknn=True)
test = graph_pp(test, bbknn=True)

del adata

computing PCA
    with n_comps=100
    finished (0:03:24)
computing batch balanced neighbors
	finished (0:00:14)
computing PCA
    with n_comps=100


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:294: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['distances'] = bbknn_out[0]
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:295: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'] = bbknn_out[1]


    finished (0:01:54)
computing batch balanced neighbors
	finished (0:00:02)
computing PCA
    with n_comps=100


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:294: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['distances'] = bbknn_out[0]
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:295: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'] = bbknn_out[1]


    finished (0:01:52)
computing batch balanced neighbors
	finished (0:00:02)


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:294: FutureWarning: This location for 'distances' is deprecated. It has been moved to .obsp[distances], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['distances'] = bbknn_out[0]
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/bbknn/__init__.py:295: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  adata.uns['neighbors']['connectivities'] = bbknn_out[1]


## Encoding 

Select tasks for prediction

1. yctype 
2. scv2_10+ (already encoded)
3. yinftime (infection + condition)


In [5]:
# encode ctype 
ctype_encoder = {v:i for i,v in enumerate(tdata.obs['ctype'].unique())}
tdata.obs['yctype'] = tdata.obs['ctype'].map(ctype_encoder)
val.obs['yctype'] = val.obs['ctype'].map(ctype_encoder)
test.obs['yctype'] = test.obs['ctype'].map(ctype_encoder)

# encode inftime
tdata.obs['inf_timepoint'] = tdata.obs['scv2_10+'].astype(str) + tdata.obs['Condition'].astype(str).apply(lambda x: '_{}'.format(x))
val.obs['inf_timepoint'] = val.obs['scv2_10+'].astype(str) + val.obs['Condition'].astype(str).apply(lambda x: '_{}'.format(x))
test.obs['inf_timepoint'] = test.obs['scv2_10+'].astype(str) + test.obs['Condition'].astype(str).apply(lambda x: '_{}'.format(x))

gt_encoder = {v:i for i,v in enumerate(tdata.obs['inf_timepoint'].unique())}
tdata.obs['yinftime'] = tdata.obs['inf_timepoint'].map(gt_encoder)
val.obs['yinftime'] = val.obs['inf_timepoint'].map(gt_encoder)
test.obs['yinftime'] = test.obs['inf_timepoint'].map(gt_encoder)


In [8]:
# create dictionary
def dictthat(AnnData, gene_ranger=True):
    """Prep dictionary for export.
    
    If gene_ranger, divide by zero can occur for 
    non-expressing genes. Thus, will floor those
    to 0.
    
    NOTE: customization re:y to predict is highly
    dependent on user input. ERGO, modify this 
    
    Arguments:
        AnnData (sc.AnnData): with graph stuff
        
    Returns:
        dict
    """
    if gene_ranger:
        # each gene in [0,1], divide by zeros to 0
        minimum = AnnData.X.min(axis=0)
        maximum = AnnData.X.max(axis=0)
        num = AnnData.X - minimum.todense()
        denom =  (maximum - minimum).todense()
        xhat = np.divide(num, denom, out=np.zeros_like(num), where=denom!=0) 
    else:
        # matrix in [0,1]
        xhat = (AnnData.X - AnnData.X.min()) / (AnnData.X.max() - AnnData.X.min())
        
    

    gdata = {'X':xhat,
             'adj':AnnData.uns['neighbors']['connectivities']+sparse.diags([1]*AnnData.shape[0], format='csr'),
             'feature_names':AnnData.var_names.to_list()}
    gdata['cell_id'] = AnnData.obs.index.to_list()
    for col in AnnData.obs.columns:
        gdata[col] = AnnData.obs[col].to_list()
    
    return gdata

gdata_train = dictthat(tdata)
gdata_val = dictthat(val)
gdata_test  = dictthat(test)



/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.


In [9]:
# export
def pklthat(gdata, fname, fpath=pdfp): 
    with open(os.path.join(fpath,fname),'wb') as f :
        pickle.dump(gdata, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()

pklthat(gdata_train, 'hbec_train_200528.pkl')
pklthat(gdata_val, 'hbec_val_200528.pkl')
pklthat(gdata_test, 'hbec_test_200528.pkl')

# clean
if False:
    del tdata, test, gdata_train, gdata_test

In [11]:
def loadpkl(filename):
    with open (filename, 'rb') as f:
        temp = pickle.load(f)
        f.close()
    return temp

def add_batch(filename, date='200529'):
    gdata = loadpkl(filename)
    batch_encoder = {v:i for i,v in enumerate(np.unique(gdata['batch']))}
    gdata['ybatch'] = list(map(batch_encoder.get, gdata['batch']))
    pklthat(gdata, '{}_{}.pkl'.format(os.path.split(filename)[1].split('_20')[0], date))
    del gdata
    print('\nBatch added and pkl saved:\n  {}_{}.pkl'.format(os.path.split(filename)[1].split('_20')[0], date))
    

In [12]:
add_batch(os.path.join(pdfp,'hbec_train_200528.pkl'))
add_batch(os.path.join(pdfp,'hbec_val_200528.pkl'))
add_batch(os.path.join(pdfp,'hbec_test_200528.pkl'))

Batch added and pkl saved:
  hbec_train_200529.pkl
Batch added and pkl saved:
  hbec_val_200529.pkl
Batch added and pkl saved:
  hbec_test_200529.pkl
